In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/taxi-trips-2023/Taxi_Trips_-_2023.csv
/kaggle/input/taxi-trips-2022-v-2/Taxi_Trips_-_2022.csv


In [2]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285413 sha256=eacb336131854ccdd3767bfbf67833aac867c457e88d48b7df286c67bb41bc4f
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [58]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
import seaborn as sns

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import StringType, ShortType, IntegerType, TimestampType, LongType, StructType, StructField, DoubleType
from pyspark.ml.evaluation import RegressionEvaluator

from pandas.tseries.holiday import USFederalHolidayCalendar

from statsmodels.tsa.seasonal import seasonal_decompose, DecomposeResult

from pyspark.ml.regression import LinearRegression, RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import VectorAssembler
from xgboost.spark import SparkXGBRegressor
from prophet import Prophet

from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [6]:
%matplotlib inline
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.plotting.backend = "plotly"

## Описание данных

- Trip ID - A unique identifier for the trip.
- Taxi ID - A unique identifier for the taxi.
- Trip Start Timestamp - When the trip started, rounded to the nearest 15 minutes.
- Trip End Timestamp - When the trip ended, rounded to the nearest 15 minutes.
- Trip Seconds - Time of the trip in seconds.
- Trip Miles - Distance of the trip in miles.
- Pickup Census Tract - The Census Tract where the trip began. For privacy, this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.
- Dropoff Census Tract - The Census Tract where the trip ended. For privacy, this Census Tract is not shown for some trips. This column often will be blank for locations outside Chicago.
- Pickup Community Area - The Community Area where the trip began. This column will be blank for locations outside Chicago.
- Dropoff Community Area -The Community Area where the trip ended. This column will be blank for locations outside Chicago.
- Fare - The fare for the trip.
- Tips -The tip for the trip. Cash tips generally will not be recorded.
- Tolls -The tolls for the trip.
- Extras - Extra charges for the trip.
- Trip Total - Total cost of the trip, the total of the previous columns.
- Payment Type - Type of payment for the trip.
- Company -The taxi company.

# 1. Загружаем и преобразуем данные

**Источник данных**

[][https://data.cityofchicago.org/Transportation/Taxi-Trips-2022/npd7-ywjz]

[][https://data.cityofchicago.org/Transportation/Taxi-Trips-2023/e55j-2ewb]

In [7]:
spark = SparkSession.builder.appName('taxi').getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/02 09:35:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
print('Spark Version: {}'.format(spark.version))

In [ ]:
file_path_1 = '/kaggle/input/taxi-trips-2022-v-2/Taxi_Trips_-_2022.csv'
file_path_2 = '/kaggle/input/taxi-trips-2023/Taxi_Trips_-_2023.csv'

df_1 = spark.read.csv(file_path_1,
                    header = True,
                    inferSchema = True,
                    nanValue = '')

# df_2 = spark.read.csv(file_path_2,
#                     header = True,
#                     inferSchema = True,
#                     nanValue = '')

In [ ]:
df = df_1

**Размер датафрейма**

In [11]:
(df.count(), len(df.columns))

(6382425, 23)

## 1.1. Schema

In [12]:
df.printSchema()

root
 |-- Trip ID: string (nullable = true)
 |-- Taxi ID: string (nullable = true)
 |-- Trip Start Timestamp: string (nullable = true)
 |-- Trip End Timestamp: string (nullable = true)
 |-- Trip Seconds: integer (nullable = true)
 |-- Trip Miles: double (nullable = true)
 |-- Pickup Census Tract: long (nullable = true)
 |-- Dropoff Census Tract: long (nullable = true)
 |-- Pickup Community Area: integer (nullable = true)
 |-- Dropoff Community Area: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Tips: double (nullable = true)
 |-- Tolls: double (nullable = true)
 |-- Extras: double (nullable = true)
 |-- Trip Total: double (nullable = true)
 |-- Payment Type: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Pickup Centroid Latitude: double (nullable = true)
 |-- Pickup Centroid Longitude: double (nullable = true)
 |-- Pickup Centroid Location: string (nullable = true)
 |-- Dropoff Centroid Latitude: double (nullable = true)
 |-- Dropoff Centroid 

## 1.2. Изменяем наименование колонок

In [13]:
df = df.toDF(*['_'.join(i.lower().split()) for i in df.columns])

In [14]:
df.columns

['trip_id',
 'taxi_id',
 'trip_start_timestamp',
 'trip_end_timestamp',
 'trip_seconds',
 'trip_miles',
 'pickup_census_tract',
 'dropoff_census_tract',
 'pickup_community_area',
 'dropoff_community_area',
 'fare',
 'tips',
 'tolls',
 'extras',
 'trip_total',
 'payment_type',
 'company',
 'pickup_centroid_latitude',
 'pickup_centroid_longitude',
 'pickup_centroid_location',
 'dropoff_centroid_latitude',
 'dropoff_centroid_longitude',
 'dropoff_centroid_location']

## 1.2. Проверяем на пропуски

In [20]:
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+-------+--------------------+------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+------------+-------+------------------------+-------------------------+------------------------+-------------------------+--------------------------+-------------------------+
|trip_id|taxi_id|trip_start_timestamp|trip_end_timestamp|trip_seconds|trip_miles|pickup_census_tract|dropoff_census_tract|pickup_community_area|dropoff_community_area|fare|tips|tolls|extras|trip_total|payment_type|company|pickup_centroid_latitude|pickup_centroid_longitude|pickup_centroid_location|dropoff_centroid_latitude|dropoff_centroid_longitude|dropoff_centroid_location|
+-------+-------+--------------------+------------------+------------+----------+-------------------+--------------------+---------------------+----------------------+----+----+-----+------+----------+------------+-------+----------------------

## 1.3. Изменяем тип данных

In [21]:
df = df.withColumn('trip_start_timestamp', F.to_timestamp(F.col('trip_start_timestamp'), 'MM/dd/yyyy hh:mm:ss a'))
df = df.withColumn('trip_end_timestamp', F.to_timestamp(F.col('trip_end_timestamp'), 'MM/dd/yyyy hh:mm:ss a'))
df = df.withColumn("pickup_community_area", F.col("pickup_community_area").cast("int"))
df = df.withColumn("dropoff_community_area",F. col("dropoff_community_area").cast("int"))

## 1.4. Создаем дополнительные фитчи

In [22]:
df.limit(5).toPandas()

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_census_tract,dropoff_census_tract,pickup_community_area,dropoff_community_area,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,pickup_centroid_location,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,bcfa19f2539021c054809d4c3993d226996ae095,368ce5511598af2cc07efdb68067d381174fdac1d47a28...,2022-01-01,2022-01-01 00:00:00,152,0.100,NaN,NaN,NaN,NaN,3.750,0.000,0.000,0.000,3.750,Cash,Medallion Leasin,NaN,NaN,None,NaN,NaN,None
1,2aba69ff015f9ea8e7bff43cab7eddb228f34a12,449fa4909552757130d09d98ebc7770e2dd94579036b0e...,2022-01-01,2022-01-01 00:30:00,2360,17.440,NaN,NaN,NaN,8.000,47.750,0.000,0.000,5.000,52.750,Cash,Flash Cab,NaN,NaN,None,41.900,-87.633,POINT (-87.6333080367 41.899602111)
2,54d812a0b88f8f9707825261014b3563a0a60ace,f98ae5e71fdda8806710af321dce58002146886c013f41...,2022-01-01,2022-01-01 00:00:00,536,4.830,NaN,NaN,28.000,22.000,14.750,0.000,0.000,0.000,14.750,Cash,Globe Taxi,41.874,-87.664,POINT (-87.6635175498 41.874005383),41.923,-87.699,POINT (-87.6991553432 41.9227606205)
3,7125b9e03a0f16c2dfb5eaf73ed057dc51eb68ef,8eca35a570101ad24c638f1f43eecce9d0cb7843e13a75...,2022-01-01,2022-01-01 00:15:00,897,2.070,NaN,NaN,8.000,32.000,9.750,0.000,0.000,1.500,11.250,Cash,Sun Taxi,41.900,-87.633,POINT (-87.6333080367 41.899602111),41.879,-87.625,POINT (-87.6251921424 41.8788655841)
4,f1a650ee419b4e52d766432e6f86eca3457bfb37,e2d8418fcdb061eee0a4318fba0a6a1200aaff0143feb0...,2022-01-01,2022-01-01 00:30:00,2200,2.480,NaN,NaN,8.000,32.000,9.360,2.140,0.000,0.000,11.500,Mobile,Chicago Independents,41.900,-87.633,POINT (-87.6333080367 41.899602111),41.879,-87.625,POINT (-87.6251921424 41.8788655841)


**Минимальная дата и максимальная дата**

In [23]:
t = df.agg(F.min('trip_start_timestamp').alias('min_date'), F.max('trip_start_timestamp').alias('max_date'))
t.show()

+-------------------+-------------------+
|           min_date|           max_date|
+-------------------+-------------------+
|2022-01-01 00:00:00|2022-12-31 23:45:00|
+-------------------+-------------------+



In [24]:
min_start_date = t.head()[0]
max_start_date = t.head()[1]

In [25]:
min_start_date, max_start_date

(datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 12, 31, 23, 45))

## 1.4. Drop unnesassery columns

In [26]:
columns_to_drop = [ 'trip_id', 'taxi_id', 'pickup_centroid_latitude', 'pickup_centroid_longitude',
                   'pickup_centroid_longitude', 'pickup_centroid_longitude', 'pickup_centroid_longitude',
                   'pickup_centroid_longitude', 'dropoff_centroid_longitude', 'dropoff_centroid_location',
                   'pickup_centroid_location', 'pickup_centroid_location', 'dropoff_centroid_latitude',
                    'pickup_census_tract', 'dropoff_census_tract', 'company', 'payment_type', 'fare', 'tips',
                   'tolls', 'extras','trip_total']
df = df.drop(*columns_to_drop)

In [27]:
df.show(10)

+--------------------+-------------------+------------+----------+---------------------+----------------------+
|trip_start_timestamp| trip_end_timestamp|trip_seconds|trip_miles|pickup_community_area|dropoff_community_area|
+--------------------+-------------------+------------+----------+---------------------+----------------------+
| 2022-01-01 00:00:00|2022-01-01 00:00:00|         152|       0.1|                 null|                  null|
| 2022-01-01 00:00:00|2022-01-01 00:30:00|        2360|     17.44|                 null|                     8|
| 2022-01-01 00:00:00|2022-01-01 00:00:00|         536|      4.83|                   28|                    22|
| 2022-01-01 00:00:00|2022-01-01 00:15:00|         897|      2.07|                    8|                    32|
| 2022-01-01 00:00:00|2022-01-01 00:30:00|        2200|      2.48|                    8|                    32|
| 2022-01-01 00:00:00|2022-01-01 00:15:00|        1256|     13.29|                   76|                

## 1.5. Заполняем пропуски

In [28]:
df = df.na.fill(99,subset=["pickup_community_area"])
df = df.na.fill(98,subset=["dropoff_community_area"])

### 1.6. Посмотрим на количество районов

In [29]:
df.select(F.countDistinct("pickup_community_area")).show()

+-------------------------------------+
|count(DISTINCT pickup_community_area)|
+-------------------------------------+
|                                   78|
+-------------------------------------+



In [30]:
df.select("pickup_community_area").distinct().show()

+---------------------+
|pickup_community_area|
+---------------------+
|                   31|
|                   65|
|                   53|
|                   34|
|                   28|
|                   76|
|                   27|
|                   26|
|                   44|
|                   12|
|                   22|
|                   47|
|                    1|
|                   52|
|                   13|
|                    6|
|                   16|
|                    3|
|                   40|
|                   20|
+---------------------+
only showing top 20 rows



## 1.6. Анализ целевой переменной

### 1.6.1 Количество заказов по районам

In [31]:
t = df.groupBy("pickup_community_area").count().toPandas()
t = t.sort_values(by='count',ascending=False)
t['pickup_community_area'] = t['pickup_community_area'].astype('str')


fig = px.bar(t, y='pickup_community_area', x='count',
              title='Количество заказов по районам',
              labels={
                     "pickup_community_area": "Район",
                     "count": "Количество"
                 },
              width=1200,
              height=700,
              orientation='h')
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.show()

По количеству заказов разброс очень большой. Есть районы с большим количество есть с маленьким.

### 1.6.1 Количество заказов по дню по всем районам[](http://)

In [32]:
# t = df.groupby(F.date_trunc('day', F.col('trip_start_timestamp')).alias('start_date')) \
#                                                      .agg(F.count('*').alias('cnt_target')) \
#                                                      .orderBy('start_date') \
#                                                      .toPandas()

# t = t.set_index('start_date')


# fig = px.line(t, t.index, y="cnt_target",
#               title='Количество заказов',
#               labels={
#                      "start_date_hour": "Дата час",
#                      "cnt_target": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

Видим тренд к лету увеличивается количестов заказов, а также сезонность по неделям

### 1.6.3 Количество заказов по районам

In [33]:
# t = df.groupBy(F.date_trunc('day', F.col('trip_start_timestamp')).alias('date'))\
#                                                                  .agg(F.count('*').alias('cnt_target')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

# fig = px.line(t, t.index, y="cnt_target",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "cnt_target": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

### 1.4.3 По районам

In [34]:
# t = df.groupBy('pickup_community_area', F.date_trunc('day', F.col('trip_start_timestamp')).alias('date'))\
#                                                                  .agg(F.count('*').alias('count')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

# fig = px.line(t, t.index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               color = 'pickup_community_area',
#               width=1400,
#               height=600)
# fig.show()

Распределение по районам приблизительно одинаковые

Посмотрим на маленькие районы по которым мало данных:

In [35]:
# s = set([11, 53, 5, 2, 3])

# fig = px.line(t[t.pickup_community_area.isin(s)], t[t.pickup_community_area.isin(s)].index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               color = 'pickup_community_area',
#               width=1400,
#               height=600)
# fig.show()

### 1.6.3. Количество заказов по месяцам по районам

In [36]:
# t = df.groupBy('pickup_community_area', F.month(F.col("trip_start_timestamp")).alias('month')) \
#                                                                      .agg(F.count('*').alias('count')) \
#                                                                      .toPandas().set_index('month') \
#                                                                      .sort_values(by=['month', 'count'],ascending=[True, True])

# fig = px.bar(t, y=t.index, x='count',
#               title='Количество заказов по месяцам',
#               labels={
#                      "count": "Количество",
#                      "month": "Месяц"
#                  },
#               width=1200,
#               height=600,
#               color='pickup_community_area',
#               orientation='h')
# fig.update_layout(yaxis=dict(autorange="reversed"))
# fig.show()

## 1.6.1 Недельная сезонность

In [37]:
# t = df.groupBy(F.date_trunc('day', F.col('trip_start_timestamp')).alias('date'))\
#                                                                  .agg(F.count('*').alias('count')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

In [38]:
# t = df.filter(F.month(F.col("trip_start_timestamp")) == 4).groupBy(F.date_trunc('day', F.col('trip_start_timestamp'))
#                                                                  .alias('date'))\
#                                                                  .agg(F.count('*').alias('count')) \
#                                                                  .orderBy('date') \
#                                                                  .toPandas()

# t = t.set_index('date')

In [39]:
# fig = px.line(t, t.index, y="count",
#               title='Количество заказов в апреле',
#               labels={
#                      "date": "Дата",
#                      "count": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

Видим еженедельную сезонность

In [40]:
# #функция отрисовки
# def plot_seasonal_decompose(result:DecomposeResult, dates:pd.Series=None, title:str="Seasonal Decomposition"):
#     x_values = dates if dates is not None else np.arange(len(result.observed))
#     return (
#         make_subplots(
#             rows=4,
#             cols=1,
#             subplot_titles=["Observed", "Trend", "Seasonal", "Residuals"],
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.observed, mode="lines", name='Observed'),
#             row=1,
#             col=1,
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.trend, mode="lines", name='Trend'),
#             row=2,
#             col=1,
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.seasonal, mode="lines", name='Seasonal'),
#             row=3,
#             col=1,
#         )
#         .add_trace(
#             go.Scatter(x=x_values, y=result.resid, mode="lines", name='Residual'),
#             row=4,
#             col=1,
#         )
#         .update_layout(
#             height=900, title=f'<b>{title}</b>', margin={'t':100}, title_x=0.5, showlegend=False
#         )
#     )

In [41]:
# decomposition = seasonal_decompose(t['count'], model='additive')
# fig = plot_seasonal_decompose(decomposition, dates=t.index)
# fig.show()

### 1.6.1 Количество заказов в течение недели

In [42]:
# t = df.filter((F.month(F.col("trip_start_timestamp")) == 3) & (F.date_format(F.col("trip_start_timestamp"), "W")==2)) \
#                                 .groupBy(F.date_trunc('hour', F.col('trip_start_timestamp')).alias('hour')) \
#                                 .agg(F.count('*').alias('count')) \
#                                 .orderBy('hour') \
#                                 .toPandas().set_index('hour')

In [43]:
# fig = px.line(t, t.index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

In [44]:
# decomposition = seasonal_decompose(t['count'], model='additive')
# fig = plot_seasonal_decompose(decomposition, dates=t.index)
# fig.show()

Есть дневная сезонность и тренд внутри недели на повышение и тренды внутри дня

### Количество заказов в течение месяца

In [45]:
# df.groupby('trip_start_timestamp', F.window('trip_start_timestamp','1 day')) \
#                                 .agg(F.count('*').alias('count')) \
#                                 .orderBy('window') \
#                                 .show(10)

In [46]:
# t = df.groupby(F.window('trip_start_timestamp','1 day')) \
#                                 .agg(F.count('*').alias('count')) \
#                                 .orderBy('window') \
#                                 .toPandas()

In [47]:
# t.head(10)

In [48]:
# fig = px.line(t, t.index, y="count",
#               title='Количество заказов по дню недели',
#               labels={
#                      "hour_trip_start_timestamp": "Дата",
#                      "count": "Количество"
#                  },
#               width=1400,
#               height=600)
# fig.show()

## 2.5 !!! Выводы по EDA

1. Есть недельная сезонность
2. Есть дневная сезонность
3. !!! Добавить праздники US Holiday
5. 

# 4. Feature Enginiring

In [49]:
AREA = 32

## 4.1. **Создаем Range из даты начала и окончания**

In [50]:
def generate_series(start, stop, interval):
    """
    :param start  - lower bound, inclusive
    :param stop   - upper bound, exclusive
    :interval int - increment interval in seconds
    
    """

    start, stop = spark.createDataFrame(
        [(start, stop)], ("start", "stop")
    ).select(
        [F.col(c).cast("timestamp").cast("long") for c in ("start", "stop")
    ]).first()
    return spark.range(start, stop, interval).select(
        F.col("id").cast("timestamp").alias("ds")
    )

In [51]:
min_start_date, max_start_date

(datetime.datetime(2022, 1, 1, 0, 0), datetime.datetime(2022, 12, 31, 23, 45))

In [52]:
generate_series(min_start_date, max_start_date, 60 * 60).show(5)
date_range_t = generate_series(min_start_date, max_start_date, 60 * 60)

+-------------------+
|                 ds|
+-------------------+
|2022-01-01 00:00:00|
|2022-01-01 01:00:00|
|2022-01-01 02:00:00|
|2022-01-01 03:00:00|
|2022-01-01 04:00:00|
+-------------------+
only showing top 5 rows



In [53]:
date_range_t.printSchema()

root
 |-- ds: timestamp (nullable = false)



In [54]:
#check
date_range_t.toPandas().set_index('ds').index.is_monotonic_increasing

True

In [55]:
date_range_t.toPandas().head(5)

,ds
0,2022-01-01 00:00:00
1,2022-01-01 01:00:00
2,2022-01-01 02:00:00
3,2022-01-01 03:00:00
4,2022-01-01 04:00:00


## Фильтруем и создаем данные

In [65]:
def create_buckets(df_t: pyspark.sql.dataframe.DataFrame, date_range_t:  pyspark.sql.dataframe.DataFrame, area: int) -> pyspark.sql.dataframe.DataFrame :
    """
    """
    
    # filtering and creating data
    df_area = df_t.filter(df_t.pickup_community_area == area).groupby(F.date_trunc('hour', F.col('trip_start_timestamp')).alias('start_date_hour')) \
                                                                                                                         .agg(F.count('*').alias('y')) \
                                                                                                                         .orderBy('start_date_hour')
    
    #df_area = df_area.withColumn('id', F.monotonically_increasing_id())
    
    
    #Join 2 DF range and main
    df_area = date_range_t.join(df_area, date_range_t.ds == df_area.start_date_hour, "left")
    
    
    #Add features day_of_week, hour, month, week_of_month
    df_area = df_area.withColumn("day_of_week",F.dayofweek(F.col("ds")))
    df_area = df_area.withColumn("hour",F.hour(F.col("ds")))
    df_area = df_area.withColumn("month",F.month(F.col("ds")))
    df_area = df_area.withColumn("week_of_month", F.date_format(F.col("ds"), "W"))
    
    #Add sin_cos
    #df_area['sin_hour'] = np.sin(2*np.pi*df['hour']/24)
    #df_area['cos_hour'] = np.cos(2*np.pi*df['hour']/24)
    
    #Fill NA Values
    df_area = df_area.fillna(0, subset=['y'])
    
    
    #Add Holidas features
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start='2022-01-01', end='2024-01-01').to_pydatetime()
    holidays = set([str(datetime.datetime.date(i)) for i in holidays])
    holidayUDF = F.udf(lambda x: 1 if str(datetime.datetime.date(x)) in holidays else 0, ShortType())
    
    df_area = df_area.withColumn('holiday', holidayUDF(F.col('ds')))
    
    
    #Add After Holiday feature
    # w = Window().partitionBy().orderBy('start_date_hour_t')
    # df_area = df_area.withColumn("after_holiday", F.lag(F.col('holiday'), 1).over(w).cast("short"))
    
    #Add Lags
    w = Window().partitionBy().orderBy('ds')

    for i in range(1, 24 + 1):
        df_area = df_area.withColumn(f"lag_{i}", F.lag(F.col('y'), i).over(w).cast("integer"))


    w = Window().partitionBy().orderBy('ds')

    for i in range(24, 24*7 + 2, 24):
        df_area = df_area.withColumn(f"lag_{i}", F.lag(F.col('y'), i).over(w).cast("integer"))
        
    #Drop NA Columns
    df_area = df_area.drop('start_date_hour')
    df_area = df_area.na.drop()

    return df_area

In [66]:
df_a = create_buckets(df, date_range_t, 32)

In [68]:
df_a.show()

+-------------------+---+-----------+----+-----+-------------+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+
|                 ds|  y|day_of_week|hour|month|week_of_month|holiday|lag_1|lag_2|lag_3|lag_4|lag_5|lag_6|lag_7|lag_8|lag_9|lag_10|lag_11|lag_12|lag_13|lag_14|lag_15|lag_16|lag_17|lag_18|lag_19|lag_20|lag_21|lag_22|lag_23|lag_24|lag_48|lag_72|lag_96|lag_120|lag_144|lag_168|
+-------------------+---+-----------+----+-----+-------------+-------+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+
|2022-01-08 00:00:00| 12|          7|   0|    1|            2|      0|   30|   35|   29|   46|   83|  102|  155|  117|  115|    82|    71|    71|    52|    60|    61|    77|  

In [67]:
#Check
df_area.filter(F.col('y').isNull()).show()

NameError: name 'df_area' is not defined

In [ ]:
df_area.limit(5).toPandas()

### 5.1. Change data type

In [ ]:
def change_data_type(df_t):
    """
    
    """
    df_t = df_t.withColumn("y",F.col("y").cast("integer"))
    df_t = df_t.withColumn("day_of_week",F.col("day_of_week").cast("integer"))
    df_t = df_t.withColumn("hour",F.col("hour").cast("integer"))
    df_t = df_t.withColumn("month",F.col("month").cast("integer"))
    df_t = df_t.withColumn("week_of_month",F.col("week_of_month").cast("integer"))
    df_t = df_t.withColumn("holiday",F.col("holiday").cast("integer"))

    return df_t

In [ ]:
df_area = change_data_type(df_area)

In [ ]:
df_area.printSchema()

------

# Models

## 1. Prothet

In [ ]:
# train.select(F.col("y"),F.col("ds"), F.col("pickup_community_area")).show()

In [ ]:
# # Define a schema
# schema = StructType([
#                     StructField('ds', TimestampType()),
#                     StructField('y', IntegerType()),
#                     StructField('day_of_week', IntegerType()),
#                     StructField('hour', IntegerType()),
#                     StructField('pickup_community_area', IntegerType()),
#                     StructField('yhat', DoubleType()),
#                     StructField('yhat_upper', DoubleType()),
#                     StructField('yhat_lower', DoubleType()),
#                     StructField('month', IntegerType()),
#                     StructField('week_of_month', IntegerType()),
#                     StructField('holiday', IntegerType()),
#                     StructField('lag_1', IntegerType()),
#                     StructField('lag_2', IntegerType()),
#                     StructField('lag_3', IntegerType()),
#                     StructField('lag_4', IntegerType()),
#                     StructField('lag_5', IntegerType()),
#                     StructField('lag_6', IntegerType()),
#                     StructField('lag_7', IntegerType()),
#                     StructField('lag_8', IntegerType()),
#                     StructField('lag_9', IntegerType()),
#                     StructField('lag_10', IntegerType()),
#                     StructField('lag_11', IntegerType()),
#                     StructField('lag_12', IntegerType()),
#                     StructField('lag_13', IntegerType()),
#                     StructField('lag_14', IntegerType()),
#                     StructField('lag_15', IntegerType()),
#                     StructField('lag_16', IntegerType()),
#                     StructField('lag_17', IntegerType()),
#                     StructField('lag_18', IntegerType()),
#                     StructField('lag_19', IntegerType()),
#                     StructField('lag_20', IntegerType()),
#                     StructField('lag_21', IntegerType()),
#                     StructField('lag_22', IntegerType()),
#                     StructField('lag_23', IntegerType()),
#                     StructField('lag_24', IntegerType()),
#                     StructField('lag_48', IntegerType()),
#                     StructField('lag_72', IntegerType()),
#                     StructField('lag_96', IntegerType()),
#                     StructField('lag_120', IntegerType()),
#                     StructField('lag_144', IntegerType()),
#                     StructField('lag_168', IntegerType()),
# ])

In [ ]:
# # define the Pandas UDF
# @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
# def apply_model(df_in: pd.DataFrame):
#     """
    
#     """
#     model = Prophet(
#       interval_width=0.95,
#       daily_seasonality=True,
#       weekly_seasonality=True,
#       yearly_seasonality=True,
#       seasonality_mode='multiplicative'
#     )
#     # fit the model to historical dat
#     model.add_country_holidays(country_name='US')
    
#     model.fit(df_in)
    
#     # Create a data frame that lists 7 days
#     future = model.make_future_dataframe(
#       periods=0,
#       freq='H',
#       include_history=True)
    
#     # Out of sample prediction
#     future = model.predict(future)
    
#     # Create a data frame that contains store, item, y, and yhat
#     f_pd = future[['ds', 'yhat', 'yhat_upper', 'yhat_lower']]
#     st_pd = df_in[['ds', 'y', '']]
#     result_pd = f_pd.join(st_pd.set_index('ds'), on='ds', how='left')

#     result_pd['yhat'] = result_pd['yhat'].clip(lower=0)
#     result_pd['yhat_lower'] = result_pd['yhat_lower'].clip(lower=0)
#     result_pd['yhat_upper'] = result_pd['yhat_upper'].clip(lower=0)
    
#     return result_pd

In [ ]:
# train.columns

In [ ]:
# # Apply the function to all store-items
# results = train.groupby(['pickup_community_area']).apply(apply_model)

In [ ]:
# results.show()

In [ ]:
# # Apply the function to all store-items
# results = train.groupby(['pickup_community_area']).apply(apply_model)
# t = results.toPandas().set_index('ds')

In [ ]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=t.index, y=t.y,
#                     mode='lines',
#                     name='y'))
# fig.add_trace(go.Scatter(x=t.index, y=t.yhat,
#                     mode='lines',
#                     name='yhat'))
# fig.show()

## 6. Train Test Split

In [ ]:
feature_list = list(df_area.columns)
feature_list.remove('y')
feature_list.remove('ds')
feature_list

In [ ]:
featureassembler = VectorAssembler(inputCols=feature_list,
                                   outputCol="independent")

In [ ]:
output = featureassembler.transform(df_area)

In [ ]:
#feature Extraction
final_data = output.select("independent", "y", "ds")
final_data.show()

# 4. Train Test Split


In [ ]:
def train_val_test_split(df_t):
    """
    
    """
    df_t = df_t.withColumn("rank", F.percent_rank().over(Window.partitionBy().orderBy("ds")))
    #df_t = df_t.withColumn('pickup_community_area', F.lit(area))
    train = df_t.filter(F.col("rank") <= 0.8).drop('rank')
    val = df_t.filter(F.col("rank") > 0.8).drop('rank')
   # test = df_t.filter(F.col("rank") > 0.9).drop('rank')
    test = val.tail(1)

    return train, val

In [ ]:
train, val = train_val_test_split(final_data)

In [ ]:
(train.count(), len(train.columns)), (val.count(), len(val.columns))

# LinearRegression

In [ ]:
model_l_reg = LinearRegression(featuresCol='independent',  maxIter = 100, 
                               labelCol='y', regParam = 0.4, elasticNetParam = 0.1)

In [ ]:
model_l_reg = model_l_reg.fit(train)

In [ ]:
model_l_reg.coefficients, model_l_reg.intercept

In [ ]:
pred = model_l_reg.evaluate(val)
pred.predictions.show()

In [ ]:
pred = model_l_reg.evaluate(train)
pred.predictions.show()

In [ ]:
print(pred.meanAbsoluteError)

### RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(featuresCol="independent",
                            labelCol="y",
                            maxDepth = 12,
                            subsamplingRate = 0.8)
model = rfr.fit(train)

In [ ]:
predict = model.transform(val)
predict.show()

# Spark XGBRegressor

In [ ]:
SparkXGBClassifier_def = SparkXGBRegressor(max_depth=6,
                                  features_col='independent',
                                  label_col='y')

In [ ]:
classifier_model = SparkXGBClassifier_def.fit(train)

In [ ]:
predict = classifier_model.transform(val)
predict.show()

In [ ]:
def mape(y_true, y_pred):
    mape = mean_absolute_percentage_error(np.array([y_true]), np.array([y_pred]))
    return float(mape)

mape_cols = F.udf(mape, DoubleType())


def mae(y_true, y_pred):
    mape = mean_absolute_error(np.array([y_true]), np.array([y_pred]))
    return float(mape)

mae_cols = F.udf(mae, DoubleType())

In [ ]:
predict = predict.withColumn('mape', mape_cols('y', 'prediction'))
predict = predict.withColumn('mae', mae_cols('y', 'prediction'))

In [ ]:
predict.show()

In [ ]:
t = predict.toPandas().set_index('ds')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t.index, y=t.y,
                    mode='lines',
                    name='y',
                    line=dict(color='blue', width=2,
                              )
                    ))
fig.add_trace(go.Scatter(x=t.index, y=t.prediction,
                    mode='lines',
                    name='prediction',
                    line=dict(color='red', width=2, dash='dot')
                    ))
fig.show()

In [ ]:
predict.limit(300).select(F.mean('mape')).show()

In [ ]:
300/24

In [ ]:

a=spark.createDataFrame([(101, 1, 16)], ['ID', 'A', 'B'])
a.show()
a.withColumn('Result', sum_cols('A', 'B')).show()

In [ ]:
a.withColumn('Result', sum_cols('A', 'B')).show()

### MAPE

In [ ]:
@pandas_udf(final_schema, PandasUDFType.GROUPED_MAP)
def gr_mape_val(join_df):
    
    mape = mean_absolute_percentage_error(join_df["y"], join_df["yhat"]) 
    join_df['mape'] = mape
  
  return join_df

df_apply = join_df.groupby('ID').applyInPandas(gr_mape_val, final_schema)
df_apply.show()

## 6. XGBoost

In [ ]:
# from pyspark.sql.functions import when, lit, col
# df3 = df2.withColumn("lit_value2", when((col("Salary") >=40000) & (col("Salary") <= 50000),lit("100")).otherwise(lit("200")))
# df3.show(truncate=False)